In [ ]:
# ===================================================================
# PROJETO COMPLETO (PLANO C): PREVISÃO COM DADOS REAIS DE EMOÇÕES
# Implementação Final com Fine-Tuning Geral e por Usuário
# ===================================================================

# -------------------------------------------------------------------
# PASSO 1: INSTALAÇÃO E IMPORTAÇÕES
# -------------------------------------------------------------------
print("🚀 PASSO 1: Instalando as ferramentas necessárias...")
!pip install transformers==4.38.2 datasets==2.18.0 accelerate==0.27.2

import torch
from torch.utils.data import DataLoader, TensorDataset
from torch.optim import AdamW
from transformers import BertTokenizer, BertForSequenceClassification
from datasets import load_dataset
import pandas as pd
from tqdm.auto import tqdm
from transformers import pipeline

print("\n✅ Ferramentas instaladas e importadas com sucesso!")

# -------------------------------------------------------------------
# PASSO 2: PREPARAÇÃO DOS DADOS REAIS DE EMOÇÕES
# -------------------------------------------------------------------
print("\n🚀 PASSO 2: Preparando os dados reais de emoções...")

# Configurar o dispositivo para usar a GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Usando o dispositivo: {device}")

# Carregar o dataset de emoções
# Labels originais: 0:sadness, 1:joy, 2:love, 3:anger, 4:fear, 5:surprise
full_dataset = load_dataset("dair-ai/emotion", trust_remote_code=True)
df = full_dataset['train'].to_pandas()

# Mapear as 6 emoções para as nossas 2 classes
# Classe 0 (Crise): tristeza (0), raiva (3), medo (4)
# Classe 1 (Normal): alegria (1), amor (2), surpresa (5)
crise_labels = [0, 3, 4]
df['binary_label'] = df['label'].apply(lambda x: 0 if x in crise_labels else 1)

# Filtrar para ter um dataset balanceado para o treino
df_crise = df[df['binary_label'] == 0].sample(2000, random_state=42)
df_normal = df[df['binary_label'] == 1].sample(2000, random_state=42)
df_geral = pd.concat([df_crise, df_normal])

# Carregar o tokenizador do NeuroBERT
MODEL_NAME = "boltuix/NeuroBERT-Tiny"
tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)

# Tokenizar os textos para o treino geral
print("Tokenizando os textos para o treino geral...")
inputs = tokenizer(df_geral['text'].tolist(), padding="max_length", truncation=True, max_length=128, return_tensors="pt")
labels = torch.tensor(df_geral['binary_label'].tolist())

# Criar o DataLoader para o treino geral
general_dataset = TensorDataset(inputs['input_ids'], inputs['attention_mask'], labels)
general_dataloader = DataLoader(general_dataset, batch_size=16, shuffle=True)

print("\n✅ Dados e configuração prontos!")


# -------------------------------------------------------------------
# PASSO 3: FINE-TUNING GERAL
# -------------------------------------------------------------------
print("\n🚀 PASSO 3: Iniciando o Fine-Tuning Geral...")

model = BertForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)
model.to(device)

optimizer = AdamW(model.parameters(), lr=5e-5)
num_epochs = 1

for epoch in range(num_epochs):
    model.train()
    progress_bar = tqdm(general_dataloader, desc=f"Epoch Geral {epoch + 1}/{num_epochs}")
    for batch in progress_bar:
        b_input_ids, b_attention_mask, b_labels = [b.to(device) for b in batch]
        optimizer.zero_grad()
        outputs = model(input_ids=b_input_ids, attention_mask=b_attention_mask, labels=b_labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        progress_bar.set_postfix({'loss': loss.item()})

model.save_pretrained("./fine-tuned-general-neurobert")
print("\n✅ Fine-Tuning Geral concluído! Modelo especialista salvo.")


# -------------------------------------------------------------------
# PASSO 4: FINE-TUNING POR USUÁRIO (PERSONALIZAÇÃO)
# -------------------------------------------------------------------
print("\n🚀 PASSO 4: Iniciando o Fine-Tuning por Usuário (Personalização)...")

model_personalizado = BertForSequenceClassification.from_pretrained("./fine-tuned-general-neurobert")
model_personalizado.to(device)

# Simular dados de um usuário (pegamos 50 amostras que não foram usadas no treino geral)
user_df = df[~df.index.isin(df_geral.index)].sample(50, random_state=42)
user_inputs = tokenizer(user_df['text'].tolist(), padding="max_length", truncation=True, max_length=128, return_tensors="pt")
user_labels = torch.tensor(user_df['binary_label'].tolist())
user_dataset = TensorDataset(user_inputs['input_ids'], user_inputs['attention_mask'], user_labels)
user_dataloader = DataLoader(user_dataset, batch_size=8)

personal_optimizer = AdamW(model_personalizado.parameters(), lr=1e-5)
num_personal_epochs = 2

for epoch in range(num_personal_epochs):
    model_personalizado.train()
    progress_bar = tqdm(user_dataloader, desc=f"Epoch Pessoal {epoch + 1}/{num_personal_epochs}")
    for batch in progress_bar:
        b_input_ids, b_attention_mask, b_labels = [b.to(device) for b in batch]
        personal_optimizer.zero_grad()
        outputs = model_personalizado(input_ids=b_input_ids, attention_mask=b_attention_mask, labels=b_labels)
        loss = outputs.loss
        loss.backward()
        personal_optimizer.step()
        progress_bar.set_postfix({'loss': loss.item()})

model_personalizado.save_pretrained("./modelo-final-personalizado")
tokenizer.save_pretrained("./modelo-final-personalizado")
print("\n✅ Fine-Tuning por Usuário concluído! Modelo personalizado salvo.")


# -------------------------------------------------------------------
# PASSO 5: TESTE FINAL DO MODELO PERSONALIZADO
# -------------------------------------------------------------------
print("\n🚀 PASSO 5: Testando o modelo final com frases novas...")

pipe_final = pipeline("text-classification", model="./modelo-final-personalizado")

# Frases de teste alinhadas com as emoções
frase_crise = "I feel so scared and lonely, I don't know what to do."
frase_normal = "I am so happy and in love with my new puppy!"

pred_crise = pipe_final(frase_crise)
pred_normal = pipe_final(frase_normal)

# Dicionário de tradução para as respostas do robô
label_map = {"LABEL_0": "Crise (Tristeza/Raiva/Medo)", "LABEL_1": "Normal (Alegria/Amor/Surpresa)"}

print("\n--- RESULTADO FINAL (MODELO PERSONALIZADO) ---")
print(f"Frase de entrada: '{frase_crise}'")
print(f"Previsão do Robô: {label_map[pred_crise[0]['label']]}\n")

print(f"Frase de entrada: '{frase_normal}'")
print(f"Previsão do Robô: {label_map[pred_normal[0]['label']]}")

🚀 PASSO 1: Instalando as ferramentas necessárias...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.7/130.7 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 49.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.9/170.9 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 48.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 31.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.6 MB/s eta 0

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/16000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

...

Epoch Geral 1/1:   0%|          | 0/250 [00:00<?, ?it/s]


✅ Fine-Tuning Geral concluído! Modelo especialista salvo.

🚀 PASSO 4: Iniciando o Fine-Tuning por Usuário (Personalização)...


Epoch Pessoal 1/2:   0%|          | 0/7 [00:00<?, ?it/s]

Epoch Pessoal 2/2:   0%|          | 0/7 [00:00<?, ?it/s]


✅ Fine-Tuning por Usuário concluído! Modelo personalizado salvo.

🚀 PASSO 5: Testando o modelo final com frases novas...

--- RESULTADO FINAL (MODELO PERSONALIZADO) ---
Frase de entrada: 'I feel so scared and lonely, I don't know what to do.'
Previsão do Robô: Crise (Tristeza/Raiva/Medo)

Frase de entrada: 'I am so happy and in love with my new puppy!'
Previsão do Robô: Normal (Alegria/Amor/Surpresa)


In [ ]:
# --- CÉLULA DE TESTE INTERATIVO ---

# ▼▼▼ Escreva aqui as suas frases em inglês para testar ▼▼▼
frase_1 = "I’m feeling really happy and motivated today!"
frase_2 = "I am feeling a lot of pressure from work lately."
# ▲▲▲ Escreva aqui as suas frases em inglês para testar ▲▲▲


# O código abaixo usa o robô já treinado para fazer a previsão
previsao_1 = pipe_final(frase_1)
previsao_2 = pipe_final(frase_2)

resultado_1 = label_map[previsao_1[0]['label']]
resultado_2 = label_map[previsao_2[0]['label']]

# Mostra os resultados na tela
print(f"Frase 1: '{frase_1}'")
print(f"Previsão do Robô: {resultado_1}")
print()
print(f"Frase 2: '{frase_2}'")
print(f"Previsão do Robô: {resultado_2}")


Frase 1: 'I’m feeling really happy and motivated today!'
Previsão do Robô: Normal (Alegria/Amor/Surpresa)

Frase 2: 'I am feeling a lot of pressure from work lately.'
Previsão do Robô: Crise (Tristeza/Raiva/Medo)
